# Python implementation documentation

## Data structures

### Graph

For representing a weighted graph inside the program, we can use various data structures, including:

1. <b>Adjacency Matrix</b>: This data structure uses a two-dimensional matrix to represent the graph. The rows and columns of the matrix represent the vertices, and the values in the matrix represent the weights of the edges between the vertices.

2. <b>Adjacency List</b>: This data structure uses an array of linked lists or arrays to represent the graph. Each vertex has a corresponding list/array containing the vertices it is connected to, along with their respective weights.

3. <b>Edge List</b>: This data structure is a simple list of edges, where each edge contains the source vertex, destination vertex, and weight.

Each data structure has its own advantages and disadvantages, depending on the specific requirements. For the case of our work, we will be using the <b>Adjacency Matrix</b>. Here's an example for a graph with 3 nodes:

In [ ]:
matrix = [
    [0, 2, 1],
    [2, 0, 1],
    [1, 1, 0],
]

### Solution candidates

A candidate solution will be represented as a list of the considered nodes (cities), randomly shuffled. For example:

In [ ]:
candidate_solution = [1, 2 ,4, 6, 8, 5]

This list represents the path 1 -> 2 -> 4 -> 6 -> 8 -> 5, in the listing order.

## Utility functions

We are now going to write a set of functions that are going to help us implement the algorithm.

### Cost function

This function calculates the cost (distance) of a solution, in order to compare candidate solutions with one another:

In [ ]:
# Calculate the cost of a tour
def cost(matrix, tour):
    n = len(matrix)

    total_distance = 0
    for i in range(n - 1):
        total_distance += matrix[tour[i]][tour[i + 1]]
    total_distance += matrix[tour[-1]][tour[0]]  # Retour à la première ville
    return total_distance

### Generate random population

This function will generate a random population of solutions that will be used as a starting point for the genetic algorithm:

In [ ]:
import random

# Initialize a random population of solutions
def initialize_population(matrix, population_size):
    population = []
    for _ in range(population_size):
        individual = list(range(len(matrix)))
        random.shuffle(individual)

        while cost(matrix, individual) == 0:
            random.shuffle(individual)

        population.append(individual)
    return population

# Selection function

This function will perform the selection phase of the genetic algorithm, choosing two 'parents' of best fitness for the next generation:

In [ ]:
# Selection
def selection(matrix, population):
    fitness_values = []
    for individual in population:
        fitness_values.append(1 / cost(matrix, individual))
    sum_fitness = sum(fitness_values)
    probabilities = [fitness / sum_fitness for fitness in fitness_values]
    parents = random.choices(population, probabilities, k=2)
    return parents

## Crossover function

This function will perform the crossover operation within selected parents in order to generate the offspring for the current generation: 

In [ ]:
def crossover(matrix, parents):
    n = len(matrix)
    parent1, parent2 = parents
    child = [None] * n
    start, end = sorted(random.sample(range(n), 2))
    child[start:end] = parent1[start:end]
    remaining_cities = [city for city in parent2 if city not in child[start:end]]
    index = 0
    for i in range(n):
        if child[i] is None:
            child[i] = remaining_cities[index]
            index += 1
    return child

## Mutation function

The mutation function randomly modifies children solutions for the current generation:

In [ ]:
def mutate(matrix, individual, mutation_rate):
    n = len(matrix)
    for i in range(n):
        if random.random() < mutation_rate:
            j = random.randint(0, n - 1)
            individual[i], individual[j] = (
                individual[j],
                individual[i],
            )
    return individual

# Proper algorithm implementation
 
The following function uses the peviously defined utility functions to build a full implementation of a the genetic algorithm for solving the TSP (and VRP) 

In [ ]:
def genetic_algorithm(matrix, population_size, generations, mutation_rate):
    population = initialize_population(matrix, population_size)
    best_individual = None
    best_fitness = float("inf")

    for _ in range(generations):
        new_population = []

        while len(new_population) < population_size:
            parents = selection(matrix, population)
            child = crossover(matrix, parents)
            child = mutate(matrix, child, mutation_rate)
            new_population.append(child)

        population = new_population

        for individual in population:
            fitness = cost(matrix, individual)
            if fitness < best_fitness:
                best_fitness = fitness
                best_individual = individual

    return best_individual, best_fitness